In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
df=pd.read_csv('train.csv')

In [22]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [23]:
new_df=df.sample(30000)

In [24]:
new_df.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0


In [25]:
#checking duplicate row in a dataset
new_df.duplicated().sum()

0

In [26]:
question_df=new_df[['question1','question2']]

In [27]:
question_df.head()

,question1,question2
384072,Which book is best for Indian history and worl...,What are some optional history books for the IAS?
330758,What are some Active Noise Cancelling headphon...,"Playing RTS or FPS games like dota 2, COD, Sta..."
385676,Will Quora be blocked in China?,Will Quora be blocked in Mainland China by the...
43606,Are Japanese game shows really like how they'r...,Do Americans wear outdoor-shoes inside their h...
97443,Do you know anyone who has managed to survive ...,Who is more to blame in the 26/11 Mumbai terro...


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

#merge texts
question=list(question_df['question1'])+list(question_df['question2'])
cv=CountVectorizer(max_features=3000)
q1_arr,q2_arr=np.vsplit(cv.fit_transform(question).toarray(),2)

In [29]:
temp_df1=pd.DataFrame(q1_arr,index=question_df.index)
temp_df2=pd.DataFrame(q2_arr,index=question_df.index)
temp_df=pd.concat([temp_df1,temp_df2],axis=1)

In [30]:
temp_df.shape

(30000, 6000)

In [31]:
temp_df['is_duplicate']=new_df['is_duplicate']

In [32]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
384072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
330758,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
385676,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
43606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97443,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307877,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
154751,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
59574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
24941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [34]:
X=temp_df.iloc[:,:-1].values
y=temp_df.iloc[:,-1].values

In [35]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=41)

In [44]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()

In [45]:
rf.fit(X_train,y_train)

RandomForestClassifier()

In [47]:
y_predict=rf.predict(X_test)

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)*100

74.75

In [53]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 5.4 MB/s eta 0:00:00


In [54]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [55]:
y_pred=xgb.predict(X_test)

In [56]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)*100

72.68333333333334